In [1]:
import sasoptpy as so
import pandas as pd
import os
from math import sqrt
import random

In [2]:
w = 16
l = 12
df = pd.read_csv('../data/xt_pre_data.csv')
df['xbin'] = pd.cut(df.start_x, w, labels=False)
df['ybin'] = pd.cut(df.start_y, l, labels=False)
df['xbin_to'] = pd.cut(df.end_x, w, labels=False)
df['ybin_to'] = pd.cut(df.end_y, l, labels=False)

In [3]:
def get_matrix(values, xbins, ybins):
    indices = [(x,y) for x in range(xbins) for y in range(ybins)]
    x_cuts = values['xbin']
    y_cuts = values['ybin']
    bins = values.groupby([x_cuts, y_cuts]).size()
    return bins.reindex(indices, fill_value=0).unstack().fillna(0)

def get_transition_matrix(values, xbins, ybins):
    indices = [(x1,y1,x2,y2) for x1 in range(xbins) for y1 in range(ybins) for x2 in range(xbins) for y2 in range(ybins)]
    x_from = values['xbin']
    y_from = values['ybin']
    x_to = values['xbin_to']
    y_to = values['ybin_to']
    valid_occurences = values[values['result_name'] == 'success'].copy()
    occurence = values.groupby([x_from, y_from, x_to, y_to]).size()
    successful = valid_occurences.groupby([x_from, y_from, x_to, y_to]).size()
    occurence_2d = occurence.reindex(indices).unstack([2,3], fill_value=0).fillna(0)
    successful_2d = successful.reindex(indices).unstack([2,3], fill_value=0).fillna(0)
    # Only successful attempts?
    return successful_2d.div(occurence_2d.sum(axis=1), axis=0)

In [4]:
shots_df = get_matrix(df[df['type_name']=='shot'], w, l)
goals_df = get_matrix(df[(df['type_name']=='shot') & (df['result_name'] == 'success')], w, l)
moves_df = get_matrix(df[df['type_name'].isin(['dribble', 'pass', 'cross'])], w, l)
total_df = moves_df + shots_df

moves = moves_df / total_df
shots = shots_df / total_df
scores = (goals_df / shots_df).fillna(0)
T = get_transition_matrix(df[df['type_name'].isin(['dribble', 'pass', 'cross'])], w, l)

In [5]:
def base_model():
    model = so.Model(name='xThreatModel', session=None)
    indices = [(x,y) for x in range(w) for y in range(l)]
    xT = model.add_variables(indices, name='xT')
    model.add_constraints(
        (xT[x,y] == shots.loc[x,y] * scores.loc[x,y] + moves.loc[x,y] * so.expr_sum(T.loc[(x,y),(z,w)] * xT[z,w] for (z,w) in indices) for (x,y) in indices), name='relation')
    model.set_objective(0, name='zero', sense='N')
    model.export_mps(filename='export.mps')
    command = 'cbc export.mps solve solu solution.txt'
    # !{command}
    os.system(command)
    for v in model.get_variables():
        v.set_value(0)
    with open('solution.txt', 'r') as f:
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            model.get_variable(words[1]).set_value(float(words[2]))
    return model

In [6]:
m1 = base_model()

NOTE: Initialized model xThreatModel.


In [7]:
xT1 = m1.get_variable('xT')
so.get_solution_table(xT1)

,xT
"(0, 0)",0.001014
"(0, 1)",0.001490
"(0, 2)",0.001763
"(0, 3)",0.002306
"(0, 4)",0.002635
...,...
"(15, 7)",0.098388
"(15, 8)",0.029122
"(15, 9)",0.026838
"(15, 10)",0.025522


In [8]:
print(xT1[11,0].get_value(),xT1[11,7].get_value())

0.014957486 0.020520949


In [9]:
print(xT1[11,3].get_value(),xT1[11,4].get_value())

0.020823058 0.019536792


In [10]:
def symmetric_model():
    model = so.Model(name='xThreatModel_sym', session=None)
    indices = [(x,y) for x in range(w) for y in range(l)]
    xT = model.add_variables(indices, name='xT')
    err = model.add_variables(indices, name='error')
    err_abs = model.add_variables(indices, name='error_abs', lb=0)
    model.add_constraints(
        (xT[x,y] + err[x,y] == shots.loc[x,y] * scores.loc[x,y] + moves.loc[x,y] * so.expr_sum(T.loc[(x,y),(z,w)] * xT[z,w] for (z,w) in indices) for (x,y) in indices), name='relation')
    model.add_constraints(
        (err_abs[x,y] >= err[x,y] for (x,y) in indices), name='abs_values1')
    model.add_constraints(
        (err_abs[x,y] >= -err[x,y] for (x,y) in indices), name='abs_values2')
    model.add_constraints(
        (xT[x,y] == xT[x, l-y-1] for (x,y) in indices), name='symm_con')
    sum_err_abs = so.expr_sum(err_abs[x,y] for (x,y) in indices)
    model.set_objective(sum_err_abs, name='total_error', sense='N')
    model.export_mps(filename='export.mps')
    command = 'cbc export.mps solve solu solution.txt'
    !{command}
    # os.system(command)
    for v in model.get_variables():
        v.set_value(0)
    with open('solution.txt', 'r') as f:
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            model.get_variable(words[1]).set_value(float(words[2]))
    return model

In [11]:
m2 = symmetric_model()

NOTE: Initialized model xThreatModel_sym.
Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Nov 25 2020 

command line - cbc export.mps solve solu solution.txt (default strategy 1)
At line 1 NAME     xThreatModel_sym
At line 2 ROWS
At line 772 COLUMNS
At line 9534 RHS
At line 9550 RANGES
At line 9551 BOUNDS
At line 9936 ENDATA
Problem xThreatModel_sym has 768 rows, 576 columns and 16840 elements
Coin0008I xThreatModel_sym read with 0 errors
Presolve 575 (-193) rows, 479 (-97) columns and 11215 (-5625) elements
Perturbing problem by 0.001% of 67.409968 - largest nonzero change 0.00042036491 ( 0.00062359458%) - largest zero change 0
Optimal - objective value 0.1442393
After Postsolve, objective 0.1442393, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0.1442392957 - 568 iterations time 0.032, Presolve 0.00
Total time (CPU seconds):       0.05   (Wallclock seconds):       0.05



In [12]:
xT2 = m2.get_variable('xT')
e = m2.get_variable('error')
so.get_solution_table(xT2)

,xT
"(0, 0)",0.000854
"(0, 1)",0.001309
"(0, 2)",0.001550
"(0, 3)",0.001873
"(0, 4)",0.002278
...,...
"(15, 7)",0.094493
"(15, 8)",0.027544
"(15, 9)",0.028804
"(15, 10)",0.024240


In [13]:
print(xT2[11,0].get_value(),xT2[11,7].get_value())

0.013471589 0.017841606


In [14]:
print(xT2[11,3].get_value(),xT2[11,4].get_value())

0.017979213 0.017841606


In [15]:
def dist(c1, c2):
    return sqrt((c1-(w-1))**2 + (c2-(l-1)/2)**2)

def sym_incremental_model():
    model = so.Model(name='xThreatModel_sym_inc', session=None)
    indices = [(x,y) for x in range(w) for y in range(l)]
    xT = model.add_variables(indices, name='xT')
    err = model.add_variables(indices, name='error')
    err_abs = model.add_variables(indices, name='error_abs', lb=0)
    model.add_constraints(
        (xT[x,y] + err[x,y] == shots.loc[x,y] * scores.loc[x,y] + moves.loc[x,y] * so.expr_sum(T.loc[(x,y),(z,w)] * xT[z,w] for (z,w) in indices) for (x,y) in indices), name='relation')
    model.add_constraints(
        (err_abs[x,y] >= err[x,y] for (x,y) in indices), name='abs_values1')
    model.add_constraints(
        (err_abs[x,y] >= -err[x,y] for (x,y) in indices), name='abs_values2')
    model.add_constraints(
        (xT[x,y] == xT[x, l-y-1] for (x,y) in indices), name='symm_con')
    model.add_constraints(
        (xT[x,y] >= xT[z, w] for (x,y) in indices for (z,w) in indices if dist(x,y) < dist(z,w) and x==z), name='same_row')
    model.add_constraints(
        (xT[x,y] >= xT[z, w] for (x,y) in indices for (z,w) in indices if dist(x,y) < dist(z,w) and y==w), name='same_col')
    sum_err_abs = so.expr_sum(err_abs[x,y] for (x,y) in indices)
    model.set_objective(sum_err_abs, name='total_error', sense='N')
    model.export_mps(filename='export.mps')
    command = 'cbc export.mps presolve off solve solu solution.txt'
    !{command}
    #os.system(command)
    for v in model.get_variables():
        v.set_value(0)
    with open('solution.txt', 'r') as f:
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            model.get_variable(words[1]).set_value(float(words[2]))
    return model

In [16]:
m3 = sym_incremental_model()

NOTE: Initialized model xThreatModel_sym_inc.
Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Nov 25 2020 

command line - cbc export.mps presolve off solve solu solution.txt (default strategy 1)
At line 1 NAME     xThreatModel_sym_inc
At line 2 ROWS
At line 3172 COLUMNS
At line 14324 RHS
At line 14340 RANGES
At line 14341 BOUNDS
At line 14726 ENDATA
Problem xThreatModel_sym_inc has 3168 rows, 576 columns and 21640 elements
Coin0008I xThreatModel_sym_inc read with 0 errors
Option for presolve changed from on to off
Perturbing problem by 0.001% of 68.680282 - largest nonzero change 0.00042734516 ( 0.00062222395%) - largest zero change 0
Optimal - objective value 0.15468368
Optimal objective 0.1546836793 - 1463 iterations time 0.182
Total time (CPU seconds):       0.21   (Wallclock seconds):       0.21



In [17]:
xT3 = m3.get_variable('xT')
e3 = m3.get_variable('error')
so.get_solution_table(xT3)
for y in range(l):
    print(y, xT3[11,y].get_value())

0 0.013572736
1 0.015088821
2 0.016415626
3 0.01798555
4 0.01798555
5 0.018052086
6 0.018052086
7 0.01798555
8 0.01798555
9 0.016415626
10 0.015088821
11 0.013572736


In [18]:
with pd.option_context('display.max_rows', None):
    print(so.get_solution_table(xT1, xT2, xT3))

                xT        xT        xT
(0, 0)    0.001014  0.000854  0.000859
(0, 1)    0.001490  0.001309  0.001316
(0, 2)    0.001763  0.001550  0.001558
(0, 3)    0.002306  0.001873  0.001884
(0, 4)    0.002635  0.002278  0.002293
(0, 5)    0.002820  0.002443  0.002458
(0, 6)    0.002849  0.002443  0.002458
(0, 7)    0.002617  0.002278  0.002293
(0, 8)    0.002218  0.001873  0.001884
(0, 9)    0.002071  0.001550  0.001558
(0, 10)   0.001530  0.001309  0.001316
(0, 11)   0.001029  0.000854  0.000859
(1, 0)    0.001516  0.001346  0.001354
(1, 1)    0.001929  0.001822  0.001832
(1, 2)    0.002535  0.002351  0.002364
(1, 3)    0.003046  0.002641  0.002652
(1, 4)    0.003250  0.002792  0.002808
(1, 5)    0.003448  0.002955  0.002979
(1, 6)    0.003396  0.002955  0.002979
(1, 7)    0.003206  0.002792  0.002808
(1, 8)    0.003132  0.002641  0.002652
(1, 9)    0.002810  0.002351  0.002364
(1, 10)   0.002191  0.001822  0.001832
(1, 11)   0.001708  0.001346  0.001354
(2, 0)    0.002118  0.001

In [19]:
table1 = so.get_solution_table(xT1)
table1.index = pd.MultiIndex.from_tuples(table1.index)
table1.unstack().transpose()

0         1         2         3         4         5         6   \
xT 0   0.001014  0.001516  0.002118  0.002609  0.003312  0.004090  0.005004   
   1   0.001490  0.001929  0.002515  0.003106  0.003830  0.004558  0.005738   
   2   0.001763  0.002535  0.003022  0.003550  0.004217  0.005050  0.006364   
   3   0.002306  0.003046  0.003533  0.003784  0.004627  0.005586  0.006593   
   4   0.002635  0.003250  0.003989  0.003988  0.004779  0.005739  0.006999   
   5   0.002820  0.003448  0.003856  0.004109  0.005046  0.006026  0.007403   
   6   0.002849  0.003396  0.003981  0.004312  0.005014  0.005933  0.007255   
   7   0.002617  0.003206  0.003554  0.004145  0.004955  0.005974  0.007194   
   8   0.002218  0.003132  0.003530  0.004042  0.004772  0.005721  0.007000   
   9   0.002071  0.002810  0.003280  0.003844  0.004418  0.005403  0.006612   
   10  0.001530  0.002191  0.002824  0.003387  0.004180  0.004915  0.006056   
   11  0.001029  0.001708  0.002113  0.002898  0.003562  0.004259  0.005323   

             7         8         9         10        11        12        13  \
xT 0   0.006172  0.007615  0.009165  0.011689  0.014957  0.017677  0.018394   
   1   0.007192  0.008621  0.010392  0.012981  0.016692  0.021262  0.024580   
   2   0.007522  0.009121  0.011123  0.014026  0.018087  0.024567  0.028702   
   3   0.008364  0.009596  0.011533  0.015221  0.020823  0.025055  0.035149   
   4   0.008476  0.010310  0.012054  0.015066  0.019537  0.029322  0.070550   
   5   0.008075  0.010009  0.012394  0.014949  0.019821  0.036462  0.077001   
   6   0.008413  0.010251  0.012648  0.015160  0.021144  0.036160  0.084869   
   7   0.008668  0.010072  0.011819  0.015216  0.020521  0.030363  0.068978   
   8   0.008439  0.009827  0.011759  0.015066  0.019864  0.025601  0.038418   
   9   0.008145  0.009449  0.011309  0.014427  0.018670  0.022498  0.026263   
   10  0.007473  0.008771  0.010548  0.013224  0.016773  0.020370  0.022982   
   11  0.006588  0.007905  0.009656  0.012247  0.015053  0.017643  0.019895   

             14        15  
xT 0   0.018630  0.018813  
   1   0.023152  0.025382  
   2   0.033817  0.030980  
   3   0.043171  0.039593  
   4   0.099819  0.091523  
   5   0.190227  0.382598  
   6   0.187113  0.428257  
   7   0.131806  0.098388  
   8   0.050468  0.029122  
   9   0.028164  0.026838  
   10  0.020692  0.025522  
   11  0.018745  0.023181

In [20]:
table2 = so.get_solution_table(xT2)
table2.index = pd.MultiIndex.from_tuples(table2.index)
table2.unstack().transpose()

0         1         2         3         4         5         6   \
xT 0   0.000854  0.001346  0.001873  0.002423  0.002930  0.003590  0.004440   
   1   0.001309  0.001822  0.002217  0.002734  0.003378  0.004148  0.005065   
   2   0.001550  0.002351  0.002639  0.003101  0.003692  0.004435  0.005615   
   3   0.001873  0.002641  0.002965  0.003283  0.004030  0.004873  0.005785   
   4   0.002278  0.002792  0.003013  0.003437  0.004141  0.004989  0.006116   
   5   0.002443  0.002955  0.003415  0.003532  0.004316  0.005231  0.006454   
   6   0.002443  0.002955  0.003415  0.003532  0.004316  0.005231  0.006454   
   7   0.002278  0.002792  0.003013  0.003437  0.004141  0.004989  0.006116   
   8   0.001873  0.002641  0.002965  0.003283  0.004030  0.004873  0.005785   
   9   0.001550  0.002351  0.002639  0.003101  0.003692  0.004435  0.005615   
   10  0.001309  0.001822  0.002217  0.002734  0.003378  0.004148  0.005065   
   11  0.000854  0.001346  0.001873  0.002423  0.002930  0.003590  0.004440   

             7         8         9         10        11        12        13  \
xT 0   0.005488  0.006797  0.008192  0.010470  0.013472  0.016013  0.016873   
   1   0.006381  0.007591  0.009249  0.011662  0.015013  0.018758  0.021641   
   2   0.006653  0.008095  0.009937  0.012569  0.016343  0.020907  0.024471   
   3   0.007312  0.008487  0.010285  0.013478  0.017979  0.023376  0.033361   
   4   0.007439  0.008840  0.010465  0.013547  0.017842  0.027915  0.069179   
   5   0.007058  0.008828  0.011025  0.013549  0.018084  0.034480  0.075593   
   6   0.007058  0.008828  0.011025  0.013549  0.018084  0.034480  0.075593   
   7   0.007439  0.008840  0.010465  0.013547  0.017842  0.027915  0.069179   
   8   0.007312  0.008487  0.010285  0.013478  0.017979  0.023376  0.033361   
   9   0.006653  0.008095  0.009937  0.012569  0.016343  0.020907  0.024471   
   10  0.006381  0.007591  0.009249  0.011662  0.015013  0.018758  0.021641   
   11  0.005488  0.006797  0.008192  0.010470  0.013472  0.016013  0.016873   

             14        15  
xT 0   0.017410  0.021948  
   1   0.021414  0.024240  
   2   0.031593  0.028804  
   3   0.041142  0.027544  
   4   0.098432  0.094493  
   5   0.184984  0.381633  
   6   0.184984  0.381633  
   7   0.098432  0.094493  
   8   0.041142  0.027544  
   9   0.031593  0.028804  
   10  0.021414  0.024240  
   11  0.017410  0.021948

In [21]:
table3 = so.get_solution_table(xT3)
table3.index = pd.MultiIndex.from_tuples(table3.index)
table3.unstack().transpose()

0         1         2         3         4         5         6   \
xT 0   0.000859  0.001354  0.001884  0.002437  0.002945  0.003611  0.004460   
   1   0.001316  0.001832  0.002230  0.002750  0.003395  0.004171  0.005090   
   2   0.001558  0.002364  0.002654  0.003116  0.003714  0.004462  0.005644   
   3   0.001884  0.002652  0.002984  0.003303  0.004056  0.004904  0.005822   
   4   0.002293  0.002808  0.003027  0.003460  0.004171  0.005027  0.006161   
   5   0.002458  0.002979  0.003434  0.003556  0.004345  0.005273  0.006517   
   6   0.002458  0.002979  0.003434  0.003556  0.004345  0.005273  0.006517   
   7   0.002293  0.002808  0.003027  0.003460  0.004171  0.005027  0.006161   
   8   0.001884  0.002652  0.002984  0.003303  0.004056  0.004904  0.005822   
   9   0.001558  0.002364  0.002654  0.003116  0.003714  0.004462  0.005644   
   10  0.001316  0.001832  0.002230  0.002750  0.003395  0.004171  0.005090   
   11  0.000859  0.001354  0.001884  0.002437  0.002945  0.003611  0.004460   

             7         8         9         10        11        12        13  \
xT 0   0.005509  0.006822  0.008230  0.010538  0.013573  0.016138  0.016986   
   1   0.006407  0.007625  0.009295  0.011708  0.015089  0.018862  0.021727   
   2   0.006692  0.008133  0.009973  0.012614  0.016416  0.021116  0.024414   
   3   0.007357  0.008535  0.010323  0.013523  0.017986  0.023332  0.033579   
   4   0.007394  0.008882  0.010502  0.013557  0.017986  0.027720  0.064943   
   5   0.007393  0.008881  0.011064  0.013557  0.018052  0.034335  0.075417   
   6   0.007393  0.008881  0.011064  0.013557  0.018052  0.034335  0.075417   
   7   0.007394  0.008882  0.010502  0.013557  0.017986  0.027720  0.064943   
   8   0.007357  0.008535  0.010323  0.013523  0.017986  0.023332  0.033579   
   9   0.006692  0.008133  0.009973  0.012614  0.016416  0.021116  0.024414   
   10  0.006407  0.007625  0.009295  0.011708  0.015089  0.018862  0.021727   
   11  0.005509  0.006822  0.008230  0.010538  0.013573  0.016138  0.016986   

             14        15  
xT 0   0.017686  0.022555  
   1   0.021727  0.025151  
   2   0.027329  0.030103  
   3   0.040124  0.040124  
   4   0.098360  0.098360  
   5   0.185903  0.429166  
   6   0.185903  0.429166  
   7   0.098360  0.098360  
   8   0.040124  0.040124  
   9   0.027329  0.030103  
   10  0.021727  0.025151  
   11  0.017686  0.022555

In [22]:
# Future Idea
def optimal_xT(xT_vals):
    model = so.Model('optimal_flow')
    indices = [(x,y) for x in range(w) for y in range(l)]
    nodes = ['source'] + indices + ['sink']
    
    use = model.add_variables(indices, name='use', vartype=so.binary)
    flow = model.add_variables(nodes, nodes, name='flow', vartype=so.binary)

    model.add_constraint(flow['source',7,5] == 1, name='initial_node')
    inflow = {(x,y): so.expr_sum(flow[i,j,x,y] for (i,j) in indices) + flow['source',x,y] for (x,y) in indices}
    outflow = {(x,y): so.expr_sum(flow[x,y,i,j] for (i,j) in indices) + flow[x,y,'sink'] for (x,y) in indices}
    model.add_constraints((inflow[x,y] == outflow[x,y] for (x,y) in indices), name='flow_balance')
    model.add_constraints((use[x,y] == inflow[x,y] for (x,y) in indices), name='use_if_flow')
    model.add_constraint(so.expr_sum(flow[x,y,'sink'] for (x,y) in indices) == 1, name='take_shot')
    model.add_constraint(so.expr_sum(use[x,y] for (x,y) in indices) == 6, name='limit_moves')
    model.add_constraints((flow[x,y,x,y] == 0 for (x,y) in indices), name='no_self_move')

    ## Replace this with probability matrix!!
    pass_ratings = so.expr_sum(T.loc[(x,y),(z,w)] * (xt_vals.iloc[z,w] - xt_vals.iloc[x,y]) * flow[x,y,z,w] for (x,y) in indices for (z,w) in indices)
    shot_rating = so.expr_sum(shots.loc[x,y] * scores.loc[x,y] * flow[x,y,'sink'] for (x,y) in indices)
    model.set_objective(-1* (shot_rating + pass_ratings), name='total_rating', sense='N')
    model.export_mps(filename='export.mps')
    command = 'cbc export.mps presolve off solve solu solution.txt'
    !{command}

xt_vals = table3.unstack()
optimal_xT(xt_vals)

NOTE: Initialized model optimal_flow.
Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Nov 25 2020 

command line - cbc export.mps presolve off solve solu solution.txt (default strategy 1)
At line 1 NAME     optimal_flow
At line 2 ROWS
At line 583 COLUMNS
At line 75307 RHS
At line 75310 RANGES
At line 75311 BOUNDS
At line 113140 ENDATA
Problem optimal_flow has 579 rows, 37828 columns and 111553 elements
Coin0008I optimal_flow read with 0 errors
Option for presolve changed from on to off
Continuous objective value is -0.436501 - 0.06 seconds
Cgl0002I 192 variables fixed
Cgl0004I processed model has 385 rows, 37055 columns (37055 integer (37055 of which binary)) and 110591 elements
Coin3009W Conflict graph built in 0.310 seconds, density: 0.132%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0038I Initial state - 6 integers unsatisfied sum - 3
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. -0.431714 iterations 987
Cbc0038I Solution found of -0.431714
Cbc